## WATEM::WhastApp

In [1]:
from omlib_super import *
from wabs import *

### Selenium Driver Provider Class
- __class__: WHP
- Method: NA
- Property: get_driver, reload

In [2]:
class WHP:

    def __init__(self, url=None):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.url = 'whatsapp.com' if url is None else url
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.ac = ActionChains(self.driver)
        print(self.driver.title)
    
    @property
    def get_driver(self):
        return self.driver
    
    @property
    def reload(self):
        wait = WebDriverWait(self.driver, 60)
        self.driver.refresh()
        search_bar_xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
        element = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpt)))
        print('reload success, whatsapp ready')
        return self
    
xx = WHP()
dd = xx.get_driver

(9) WhatsApp


### Selenium Required Functions
- sidepane_scroll(d, initial = 1, scrolls=50)
- msg_scoll_up(d):
- wtx(d, xpt, wt=1):
- ww(d, xpt, sec=2):
- wx(d, xpt, elements=None, wait=None, text=None, click=None):
- write_msg(d, xpt, text):
- msg_sender_dttm(d, xpt):
- just_move(d, xpt="(//div[@data-testid='conversation-panel-body'])[2]"):

In [3]:
def sidepane_scroll(d, initial = 1, scrolls=50):
    for i in range(1, scrolls):
        d.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial = initial + i + 1
    return initial

def msg_scoll_up(d):
    d.find_element(By.XPATH, "//div[@class='_5kRIK']").send_keys(Keys.HOME)
    tm.sleep(2)
    
def wtx(d, xpt, wt=1):
    try: return d.find_element(By.XPATH, xpt).text
    except: return None

def ww(d, xpt, sec=2):
    wait = WebDriverWait(d, sec)
    try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
    except: return None
    
def wx(d, xpt, elements=None, wait=None, text=None, click=None):
    try:
        if elements is not None:
            return d.find_elements(By.XPATH, xpt)
        elif wait is not none: 
            y = ww(d, xpt, wait)   
        else:
            y = d.find_element(By.XPATH, xpt)
        if text is not None: return y.text
        elif click is not None: 
            y.click()
            return 1
        else: return y
    except: return None
        
def write_msg(d, xpt, text):
    ac = ActionChains(d)
    ac.move_to_element(ww(d,xpt)).click().perform()
    if chr(10) not in text:
        ac.send_keys(text).send_keys(Keys.RETURN).perform()
    else:
        xx = text.split(chr(10))
        for line in xx:
            ac.send_keys(line).perform()
            ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
        ac.send_keys(Keys.RETURN).perform()

def msg_sender_dttm(d, xpt):
    elm = ww(d, xpt) if type(xpt) is str else xpt
    html = elm.get_attribute('innerHTML')
    sp = BeautifulSoup(html, "lxml")
    x = sp.find_all('div')
    for i in x:
        try:
            st = i['data-pre-plain-text']
            break
        except:pass
    else: return None, None
    try:
        split_point = st.find(']')
        sender = st[split_point+1:].replace(':','').strip(' ')
        dttm = parse(st.upper()[1:split_point])
        return dttm.strftime("%Y-%m-%d %H:%M"), sender
    except: None, None
        
def just_move(d, xpt="(//div[@data-testid='conversation-panel-body'])[2]"):
    try:
        ActionChains(d).move_to_element(ww(d, xpt, 3)).perform()
        tm.sleep(1)
        return 1
    except: 
        print('just move calling msg_scoll_up')
        return msg_scoll_up(d)

### WhatsApp Specific Functions

#### whatsapp  XPATH bulilt and global

- msgbase = "(//div[@id='main']//div[@data-testid='msg-container'])"
- msgbs = "//div[@class='ItfyB _3nbHh']"
- pnbase ="(//div[@id='pane-side']//div[@data-testid='cell-frame-container'])"
- pnbs = "//div[@class='_8nE1Y']"
- search_msg_bytext(text = [('@class','_21Ahp'), 'omi'], index=None):
- search_msg_by_sender_text(sender='omi', text=None, index=None):
- pane_chat_byname(chatname)
- pane_chat_unread_byname(chatname

In [33]:
from xbuilder import *

pane_search = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
body_chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
body_msg_nav_arrow = '/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div'
init_info = "//div[@data-testid='intro-text'][contains(.,'Use WhatsApp on up to 4 linked devices ')]"
pane_search = "//div[@id='side']//div[@data-testid='chat-list-search']"
pane_search_back ="//div[@id='side']//span[@data-testid='search']"
pane_search_cancel = "//div[@id='side']//span[contains(@data-testid,'x-alt')]"
pane_search_filter = "//div[@id='side']//button[@aria-label='Unread chats filter']"
active_chat_title = "//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']"
active_chat_search = "//div[@id='main']//span[@data-testid='search-alt']"
msg_menu = '//*[contains(concat( " ", @class, " " ), concat( " ", "_3bcLp", " " ))'

msgbase = "(//div[@id='main']//div[@data-testid='msg-container'])"
msgbs = "//div[@class='ItfyB _3nbHh']"
msg_byindex = lambda i : msgbase + '[' + str(i) + ']'

def search_msg_bytext(text = [('@class','_21Ahp'), 'omi'], index=None):
    txtm = '(' + contains_gen(contains, text,'div') 
    txtm = txtm + "[" + str(index) + "]" if type(index) is not None else txtm
    fxpt = txtm + "/ancestor::div[@data-testid='msg-container']"
    return fxpt

def search_msg_by_sender_text(sender='omi', text=None, index=None):
    txtm = "(//div[@class='ItfyB _3nbHh'])[" + starts_with_no_attr(sender) + ']'
    axes="/ancestor::div[@data-testid='msg-container']"
    if text is not None:
        for n in text:
            nd = tuple(n) if n is str else n
            txtm = contains_add(contains, txtm, nd , 'starts-with')
    xptx = txtm + '[' + str(index) + ']'  + axes if index is not None else txtm + axes
    return xptx

pnbase ="div[@data-testid='cell-frame-container'])"
pnbs = "div[@class='_8nE1Y']"
pane_chat_byname = lambda chatname: "//div[@class='y_sn4']//span" + "[contains(.,'" + chatname + "')]//ancestor::" + pnbase
pane_chat_unread_byname = lambda chatname : pane_chat_byname(chatname) + "//span[@data-testid='icon-unread-count']"

#### Section:1 (Methods):
- **select_chat**(d, chat_name)
- __chatinfo__ (d, n=None, pane_base="//div[@id='pane-side']//div[@data-testid='cell-frame-container']")
- __msginfo__ (d, n=None, base="(//div[@id='main']//div[@data-testid='msg-container'])")

In [5]:
def select_chat(d, chat_name):
    if chat_name is not None and chat_name != '':
        convtitle = "//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']"
        xx = wtx(d, convtitle)
        if xx is not None and chat_name.lower() in xx.lower(): 
            print('already selected')
            wx(d,'/html/body/div[1]/div/div/div[4]/div/div[2]/div/div[1]/span/div/span[2]', click=True)
            return 1
        else:
            if wx(d, pane_chat_byname(chat_name), click=True) == 0:
                print('chat has no communication')
                ActionChains(d).send_keys_to_element(ww(d,"/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"),chat_name).send_keys(Keys.RETURN).pause(1).perform()
            y = wtx(d, convtitle)
            print('chat body name: ', y)
            if chat_name.lower() in y.lower():
                print('chat selection successful')
                wx(d,"//div[@id='side']//span[contains(@data-testid,'x-alt')]", click=True)
                wx(d,"//div[@id='side']//button[@aria-label='Unread chats filter']", click=True)
            else: 0
    else: return 0

def wapane_chat_type(d, bs):
    xpt = bs + "//div[@data-testid='chatlist-status-v3-ring']"
    if ww(d, xpt.strip(' ')) is None: 
        return 'group'
    else: 
        return 'contact'
    
def chatinfo(d, n=None, pane_base="//div[@id='pane-side']//div[@data-testid='cell-frame-container']"):
    base = '(' + pane_base + "//div[@class='_8nE1Y'])" if '_8nE1Y' not in pane_base else pane_base
    bi = base + '[' + str(n) + ']' if n is not None else base
    bi4ctype = '(' + pane_base + ')[' + str(n) + ']' if n is not None else pane_base 
    if ww(d, bi) is None: return None
    print('pane coll xpt not none: ', bi)
    dc = {'chat_type': [wapane_chat_type(d, bi4ctype)],
        'chat_name' : [bi + "//div[@class='y_sn4']//span"],
        'last_text' : [bi + "//div[@class='vQ0w7']//span[@dir='ltr']"],
        'last_sender' : [bi + "//div[@class='vQ0w7']//span[@dir='auto']"],
        'time' : [bi + "/div[@class='y_sn4']//div[@class='Dvjym']"],
        'new_msg_notif' : [bi + "//span[@data-testid='icon-unread-count']"]}
    for k, v in dc.items():
        v = [v] if type(v) is str else v
        dc[k] = [wtx(d, j) for j in v]
    return dc

def msginfo(d, n=None, base="(//div[@id='main']//div[@data-testid='msg-container'])"):
    bi = base + '[' + str(n) + ']' if n is not None else base
    print('msgbase is being collecting:', bi)
    elm = ww(d, bi)
    if elm is None: return None
    else:
        try: ActionChains(d).move_to_element(ww(d, bi + "//div[@role='button'][contains(text(),'Read more')]")).click().perform()
        except: pass
        dttm, sender = msg_sender_dttm(d, elm)
        dc = {'sender' : [sender],
              'text' : [wtx(d, bi + "//div[@class='_21Ahp']/span[1]/span")],
              'q_sender' : [wtx(d, bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
              'q_text' : [wtx(d, bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
              'datetime' : [dttm],
              'sender_visible': [wtx(d, bi + "//div[@class='_27K43 _31p5Q']/div[1]//span[@dir='auto']")],
              'time': [wtx(d, bi + "//div[@data-testid='msg-meta']/span")]
              }
        dc['img'] = 'image' if ww(d, bi + "//div[@data-testid='image-thumb']", 1) is not None else 'NoImage'
        return dc

### Actions
- action(d, k, v):  '_sendtxt_', 'key_to_element', '_clk_','_justmove_','_hoverclick_', '_moveclick_'

In [6]:
import wa_actions as wact
  
def pending_jobs(d, action_dic, check_from_key = None):
    '''check key and return pending jobs'''
    key_found, dc = False, {}
    for k, v in action_dic.items():
        if check_from_key is not None and key_found==False:
            if k == check_from_key: 
                key_found = True
                dc = {k:v}
        else:
            dc[k] = v
    else: return dc

def rjobs(d, action_dic):
    '''check action dictionary reversely and returns key but returns 1 while last key is present'''
    for i in reversed(list(action_dic)):
        print(i, action_dic[i][0])
        elm = ww(d, action_dic[i][0])
        if elm is not None:
            if 'present' in i: return 1
            else: return i
        else: 
            print(action_dic[i][0])

### Iterator

#### The perfect iterator
- **search_in**(d, func, scroll_func, scroll_arg=None, xpt=None, last=False, chat=None)
- **msg** = search_in(d=dd, func=wm.msginfo, xpt="(//div[@id='main']//div[@data-testid='msg-container'])", scroll_func=wm.just_move, scroll_arg=None, chat='Emergency')
- **pane** = search_in(d=dd, func=wm.side_pane, scroll_func=wm.sidepane_scroll, scroll_arg=1)

In [7]:
def search_in(d, func, scroll_func, scroll_arg=None, xpt=None, last=False, chat=None):
    if chat is not None: 
        if wact.select_chat(d, chat) == 0: return "none"
    dc, i, scroll_limit = defaultdict(list), 1, 0
    while scroll_limit<10:
        if last==True: xp = func(d,'last()-'+str(i),xpt) if xpt is not None else func(d,'last()-'+str(i))
        else: xp = func(d,i,xpt) if xpt is not None else func(d,i)
        if xp is not None:
            yield xp, i, scroll_limit
            i, scroll_limit = i + 1, 0
        else:
            if scroll_limit>1:
                scroll_arg = scroll_func(d, scroll_arg) if scroll_arg is not None else scroll_func(d)
                i = 1 if scroll_arg is not None else i
            scroll_limit = scroll_limit + 1

msg = search_in(d=dd, func=msginfo, xpt="(//div[@id='main']//div[@data-testid='msg-container'])", 
               scroll_func=just_move, scroll_arg=None, chat='Emergency')

pane = search_in(d=dd, func=chatinfo, scroll_func=sidepane_scroll, scroll_arg=1)
#for i in ab:
    #dc, inx, scroll = i
    #print(dc)
    #if inx > 20: break
#print('nice job')

#### message search in chat
- search_msg_in_chat(d, chat_name, search_xpath):

In [8]:
def search_msg_in_chat(d, chat_name, search_xpath):
    no_of_items, same_elm_count, i = defaultdict(list), 0, 0
    while same_elm_count<=5:
        el = ww(d, search_xpath)
        if el is not None:
            no_elm = len(d.find_elements(By.XPATH, search_xpath))
            if no_elm == (i+1): 
                yield el
                i = i + 1
            else:
                for m in range(i+1, item):
                    yield m
                    i = i + 1
        else:
            print(just_move(d))
        pre_no_of_items = no_of_items
        no_of_items = len(d.find_elements(By.XPATH, "(//div[@id='main']//div[@data-testid='msg-container'])"))
        if no_of_items == pre_no_of_items: same_elm_count = same_elm_count + 1
        else: same_elm_count = 0
    print(same_elm_count, ' is 3/3+ so, seems there is no more content to scroll')

### BS4

In [8]:
def bs_attr_inner_text(html, tag='div', attr='data-pre-plain-text'):
    elm = ww(d, xpt) if type(xpt) is str else xpt
    html = elm.get_attribute('innerHTML')
    sp = BeautifulSoup(html, "lxml")
    x = sp.find_all('div')
    for i in x:
        try: return i['data-pre-plain-text']
        except: pass
    else:
        print(attr , ' --> not found')


### WIP Development

In [9]:
#fw_selections(d, fw_actions,contacts_and_xpath=[('omi','xpath')]):

In [52]:
def x_action(d, k, v):
    print('from x_action')
    ac = ActionChains(d)
    if k in '_sendtxt_': write_msg(d, v[0], v[1])
    elif k in 'key_to_element': ac.send_keys_to_element(ww(d,v[0]), v[1]).send_keys(Keys.RETURN).perform()
    elif k in '_clk_': ActionChains(d).click(ww(d, v[0])).perform()
    elif k in '_justmove_': ac.move_to_element(ww(d, v[0])).perform()
    elif k in '_hoverclick_': 
        ac.move_to_element(ww(d, v[0])).pause(.5).perform()
        ac.click(ww(d, v[1])).perform()
    elif k in '_moveclick_': ac.move_to_element(ww(d, v[0])).pause(1).click().perform()
    else: 
        print('action print from else')
        return None
    
forward = lambda d, chatname, search_xpath, text : \
        {'func': {
            'function': select_chat(d, chatname),
            'move_func': search_xpath
            },
         'clicking':{
            'clk_1': ["//span[@data-testid='down-context']"],
            'clk_2': ["//ul//li[contains(.,'Forward message')]"],
            'clk_3': ["//button[@title='Forward message']"],
            'is_present':["//h1[text()='Forward message to']"]
         }}

In [53]:
pane_chat_byname('Emergency SOC')

"//div[@class='y_sn4']//span[contains(.,'Emergency SOC')]//ancestor::div[@data-testid='cell-frame-container'])"

In [54]:
x_action(dd, 'clk', ["//div[@class='y_sn4']//span[contains(.,'Emergency SOC')]//ancestor::(//div[@id='pane-side']//div[@data-testid='cell-frame-container'])"])

from x_action


In [55]:
x_action(dd, 'clk',[pane_chat_byname('Emergency SOC')])

from x_action


In [56]:
x_action(dd, 'hover', [msg_byindex('last()'),"//span[@data-testid='down-context']"])

In [57]:
x_action(dd, 'move', [msg_byindex('last()'),"//span[@data-testid='down-context']"])

"//span[@data-testid='down-context']"

from x_action


In [ ]:
waxpath